Natural Language Processing Tutorial
======

This is the tutorial of the 2024 [Mediterranean Machine Learning Summer School](https://www.m2lschool.org/) on Natural Language Processing!

This tutorial will explore the fundamental aspects of Natural Language Processing (NLP). Basic Python programming skills are expected.
Prior knowledge of standard NLP techniques (e.g. text tokenization and classification with ML) is beneficial but optional when working through the notebooks as they assume minimal prior knowledge.

This tutorial combines detailed analysis and development of essential NLP concepts via custom (i.e. from scratch) implementations. Other necessary NLP components will be developed using PyTorch's NLP library implementations. As a result, the tutorial offers deep understanding and facilitates easy usage in future applications.

## Outline

* Part I: Introduction to Text Tokenization and Classification
  *  Text Classification: Simple Classifier
  *  Text Classification: Encoder-only Transformer

* Part II: Introduction to Decoder-only Transformer and Sparse Mixture of Experts Architecture
  *  Text Generation: Decoder-only Transformer
  *  Text Generation: Decoder-only Transformer + MoE

* Part III: Introduction to Parameter Efficient Fine-tuning
  *  Fine-tuning the full Pre-trained Models
  *  Fine-tuning using Low-Rank Adaptation of Large Language Models (LoRA)

## Notation

* Sections marked as [📝] contain cells with missing code that you should complete.
* Sections marked with [📚] contain cells that you should read and modify to understand how your changes alter the obtained results.
* External resources are mentioned with [✨]. These provide valuable supplementary information for this tutorial and offer opportunities for further in-depth exploration of the topics covered.
* Sections that contain code that test the functionality of other sections are marked with [✍]. You are more that welcome to modify these sections so that you can understand code functionality.


## Libraries

This tutorial leverages [PyTorch](https://pytorch.org/) for neural network implementation and training, complemented by standard Python libraries for data processing and the [Hugging Face](https://huggingface.co/) datasets library for accessing NLP resources.

GPU access is recommended for optimal performance, particularly for model training and text generation. While all code can run on CPU, a CUDA-enabled environment will significantly speed up these processes.

## Credits

The tutorial is created by:

* [Georgios Peikos](https://www.linkedin.com/in/peikosgeorgios/)
* [Luca Herranz-Celotti](http://LuCeHe.github.io)

It is inspired by and synthesizes various online resources, which are cited throughout for reference and further reading.

## Note for Colab users

To grab a GPU (if available), make sure you go to `Edit -> Notebook settings` and choose a GPU under `Hardware accelerator`





---



# Part II: Introduction to the decoder-only Transformers architecture and Sparse Mixture of Expert

We create a decoder-only Transformer architecture from the bottom up, including a custom text tokenizer and an efficient dataset handler. We will explore all essential components of this architecture, train the model, and show its capabilities in text generation.

Then, we will enhance our base model by incorporating a gating function and implementing a sparse mixture of experts.



---



# Decoder-only Transformer Architecture

The decoder-only transformer architecture consists of multiple identical blocks stacked sequentially. Each block is composed of two main elements:
- A masked multi-head self-attention mechanism.
- A feed-forward neural network.

These components are typically encapsulated within residual connections and layer normalization. In this section, we will explore the internal structure of these blocks in greater depth and provide a practical PyTorch implementation.

![Decoder Only Architecture](https://drive.google.com/uc?id=1ksROxQxf3b7dlBUoIQggzyLeBaPO-AQn)






---



## Importing Libraries

In [ ]:
!pip install datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import math
from collections import Counter
from typing import List, Tuple, Union
from datasets import load_dataset
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 📚 Text Tokenization from Scratch

Tokenization is a fundamental step in NLP that converts raw text into a format that systems can understand and process. It enables the transformation of variable-length text sequences into fixed-size numerical representations, which is crucial for input to neural network models.

Here, we create a simple text tokenizer for basic word-level tokenization tasks.
The tokenizer can be improved so that:

1.   Methods for handling very large vocabularies (e.g., frequency thresholding)
2.   Support for n-grams or phrase detection
3.   Handle punctuation. For instance now, tokens like "word." and "word" are being treated differently.

Also, we create a testing function to showcase the codes behavior.

**✨ Additional Resources:**

*   Overview of hugging Face tokenizers [Link-huggingface](https://huggingface.co/docs/transformers/en/tokenizer_summary)

In [ ]:
class SimpleTokenizer:
    def __init__(self):
        """Initialize the tokenizer with special tokens and prepare vocabulary structures."""
        # Special tokens are used for various purposes in NLP tasks:
        # <PAD>: Used for padding sequences to a fixed length
        # <UNK>: Represents unknown words not in the vocabulary
        # <SOS>: Marks the start of a sequence
        # <EOS>: Marks the end of a sequence
        self.special_tokens = ["<PAD>", "<UNK>", "<SOS>", "<EOS>"]

        # word_to_idx: Maps words to unique integer indices
        # This is crucial for converting text into a format that neural networks can process
        self.word_to_idx = {token: idx for idx, token in enumerate(self.special_tokens)}

        # idx_to_word: The reverse mapping of word_to_idx
        # This is used for converting model outputs back into readable text
        self.idx_to_word = {idx: token for idx, token in enumerate(self.special_tokens)}

        # Counter object to keep track of word frequencies in the corpus
        self.word_count = Counter()

    def fit(self, texts: List[str]) -> None:
        """Build the vocabulary from a list of texts."""
        # Count the frequency of each word in the entire corpus
        for text in texts:
            self.word_count.update(text.split())

        # Add each unique word to the vocabulary
        # We assign a unique index to each word, which the model will use to represent words
        for word in self.word_count:
            if word not in self.word_to_idx:
                idx = len(self.word_to_idx)
                self.word_to_idx[word] = idx
                self.idx_to_word[idx] = word

    def encode(self, text: str) -> List[int]:
        """Convert a text string to a list of indices."""
        # This method is used to prepare input for the model
        # It converts each word to its corresponding index
        # If a word is not in the vocabulary, it uses the <UNK> token
        return [self.word_to_idx.get(word, self.word_to_idx["<UNK>"]) for word in text.split()]

    def decode(self, indices: List[int]) -> str:
        """Convert a list of indices back to a text string."""
        # This method is used to convert model output back into readable text
        # It maps each index back to its corresponding word
        return " ".join([self.idx_to_word.get(idx, "<UNK>") for idx in indices])

    def encode_batch(self, texts: List[str]) -> List[List[int]]:
        """Convert a batch of text strings to lists of indices."""
        return [self.encode(text) for text in texts]

    def decode_batch(self, batch_indices: List[List[int]]) -> List[str]:
        """Convert a batch of lists of indices back to text strings."""
        return [self.decode(indices) for indices in batch_indices]

    def show_vocab(self):
        """Display the vocabulary."""
        # Useful for debugging and understanding the tokenizer's state
        print("Vocabulary:")
        for word, idx in self.word_to_idx.items():
            print(f"{word}: {idx}")

    def __len__(self):
        """Return the size of the vocabulary."""
        # The vocabulary size is an important parameter for the model
        # It determines the dimensionality of the model's output layer
        return len(self.word_to_idx)

### ✍ Testing the Tokenizer

This testing function shows examples of text tokenization presenting also extreme use cases.

In [ ]:
def test_tokenizer():
    print("\nTesting SimpleTokenizer")
    print("=" * 30)

    # Sample texts
    texts = [
        "The quick brown fox jumps over the lazy dog.",
        "Pack my box with five dozen liquor jugs!",
        "How vexingly quick daft zebras jump!",
        "This is a sentence with some punctuation, including commas.",
        "This text contains an unknown word: monkey",
        ""  # Empty string to test edge case
    ]

    # Initialize and fit the tokenizer
    tokenizer = SimpleTokenizer()
    tokenizer.fit(texts)

    # Display vocabulary
    tokenizer.show_vocab()
    print(f"\nVocabulary size: {len(tokenizer)}")

    # Test encoding and decoding
    print("\nEncoding and Decoding Test:")
    for text in texts:
        encoded = tokenizer.encode(text)
        decoded = tokenizer.decode(encoded)
        print(f"\nOriginal: {text}")
        print(f"Encoded : {encoded}")
        print(f"Decoded : {decoded}")
        print(f"Match   : {'✓' if text.strip().lower() == decoded.strip().lower() else '✗'}")

    # Test unknown word handling
    print("\nUnknown Word Handling Test:")
    unknown_text = "This text contains an unknown word: xylophone"
    encoded_unknown = tokenizer.encode(unknown_text)
    decoded_unknown = tokenizer.decode(encoded_unknown)
    print(f"Original: {unknown_text}")
    print(f"Encoded : {encoded_unknown}")
    print(f"Decoded : {decoded_unknown}")

    # Test special tokens
    print("\nSpecial Tokens Test:")
    special_text = "< SOS > This is a test sentence <EOS>"
    encoded_special = tokenizer.encode(special_text)
    decoded_special = tokenizer.decode(encoded_special)
    print(f"Original: {special_text}")
    print(f"Encoded : {encoded_special}")
    print(f"Decoded : {decoded_special}")

    # Test case sensitivity
    print("\nCase Sensitivity Test:")
    case_text = "The Quick Brown Fox"
    encoded_case = tokenizer.encode(case_text)
    decoded_case = tokenizer.decode(encoded_case)
    print(f"Original: {case_text}")
    print(f"Encoded : {encoded_case}")
    print(f"Decoded : {decoded_case}")

print("\nChecking the tokenizer's outputs")
test_tokenizer()

## 📚 TextDataset: Efficient Text Processing

The TextDataset class is a crucial component in preparing text data for deep learning models, implementing a sliding window approach that allows processing of variable-length texts while maintaining context.

This class bridges the gap between raw text data and the input requirements of neural networks, handling tasks such as tokenization, padding, and attention mask generation, which are essential for training effective sequence models like Transformers.

**✨ Additional Resources:**

*   Padding and truncation [Link-huggingface](https://huggingface.co/docs/transformers/en/pad_truncation)


In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts: List[str], tokenizer: SimpleTokenizer, max_length: int, overlap: int = 50):
        """
        Initialize the TextDataset with sliding window functionality.

        Args:
            texts (List[str]): List of input texts.
            tokenizer (SimpleTokenizer): Tokenizer object for encoding texts.
            max_length (int): Maximum length of encoded sequences.
            overlap (int): Number of overlapping tokens between windows.
        """
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.overlap = overlap
        self.data = []
        self.attention_masks = []
        self.document_map = []  # Maps each window to its original document
        self.original_texts = texts  # Store original texts

        for doc_idx, text in enumerate(texts):
            tokens = self.tokenizer.encode(text)
            windows = self.create_sliding_windows(tokens)

            for window in windows:
                attention_mask = [1] * len(window)  # 1 for real tokens

                # Pad if necessary
                if len(window) < max_length:
                    padding_length = max_length - len(window)
                    window = window + [self.tokenizer.word_to_idx["<PAD>"]] * padding_length
                    attention_mask = attention_mask + [0] * padding_length  # 0 for padding in attention mask

                self.data.append(window)
                self.attention_masks.append(attention_mask)
                self.document_map.append(doc_idx)

    def create_sliding_windows(self, tokens: List[int]) -> List[List[int]]:
        """
        Create sliding windows from a list of tokens.

        Args:
            tokens (List[int]): List of token ids.

        Returns:
            List[List[int]]: List of token windows.
        """
        windows = []
        # Calculate stride: how many tokens to move for each new window
        # -1 accounts for the added <SOS> token at the start of each window
        stride = self.max_length - self.overlap - 1

        for start in range(0, len(tokens), stride):
            # Create a window starting with <SOS> token
            window = [self.tokenizer.word_to_idx["<SOS>"]] + tokens[start:start + self.max_length - 1]
            if len(window) < self.max_length:
                # This is the last window, add <EOS> token
                window.append(self.tokenizer.word_to_idx["<EOS>"])
            windows.append(window)

        return windows

    def get_original_document(self, doc_idx: int) -> str:
        """Retrieve the original document text."""
        if 0 <= doc_idx < len(self.original_texts):
            return self.original_texts[doc_idx]
        else:
            raise IndexError(f"Document index {doc_idx} is out of range.")

    def get_document_length(self, doc_idx: int) -> int:
        """Get the number of tokens in the original document."""
        if 0 <= doc_idx < len(self.original_texts):
            return len(self.tokenizer.encode(self.original_texts[doc_idx]))
        else:
            raise IndexError(f"Document index {doc_idx} is out of range.")

    def window_to_document_position(self, window_idx: int, token_idx: int) -> Tuple[int, int]:
        """Map a position in a window back to its position in the original document."""
        if 0 <= window_idx < len(self.data):
            doc_idx = self.document_map[window_idx]
            doc_windows = self.get_document_windows(doc_idx)
            # Find which window of the document this is
            relative_window_idx = doc_windows.index(window_idx)
            # Calculate the start position of this window in the document
            window_start = relative_window_idx * (self.max_length - self.overlap - 1)
            # -1 to account for <SOS> token at the start of each window
            return doc_idx, window_start + token_idx - 1
        else:
            raise IndexError(f"Window index {window_idx} is out of range.")

    def __len__(self) -> int:
        """Get the number of windows in the dataset."""
        return len(self.data)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, int]:
        """
        Get a sample from the dataset.

        Args:
            idx (int): Index of the sample.

        Returns:
            Tuple[torch.Tensor, torch.Tensor, int]:
                A tuple containing (token_ids, attention_mask, document_index).
        """
        if 0 <= idx < len(self.data):
            # Add an extra dimension to make it batch-first (batch_size=1)
            return (torch.tensor(self.data[idx]).unsqueeze(0),
                    torch.tensor(self.attention_masks[idx]).unsqueeze(0),
                    self.document_map[idx])
        else:
            raise IndexError(f"Index {idx} is out of range.")


    def get_document_windows(self, doc_idx: int) -> List[int]:
        """
        Get all window indices for a specific document.

        Args:
            doc_idx (int): Index of the document.

        Returns:
            List[int]: List of window indices belonging to the document.
        """
        return [i for i, doc in enumerate(self.document_map) if doc == doc_idx]

### ✍ Testing the Dataset Processing

This testing function shows how the TextDataset and SimpleTokenizer classes work together.

In [ ]:
def test_sliding_window_dataset():
    print("\n--- Testing Sliding Window Dataset ---\n")

    texts = [
        "This is a short sentence.",
        "This is a much longer sentence that will be split into multiple windows to demonstrate the sliding window approach. It contains enough tokens to create at least two or three windows depending on the chosen maximum length and overlap.",
        "Another sentence of medium length that might create two windows.",
        "",  # Empty text to test edge case
        "Short."  # Very short text to test edge case
    ]

    try:
        tokenizer = SimpleTokenizer()
        tokenizer.fit(texts)

        max_length = 16
        overlap = 5
        dataset = TextDataset(texts, tokenizer, max_length, overlap)

        print(f"Dataset configuration:")
        print(f"  Max length: {max_length}")
        print(f"  Overlap: {overlap}")
        print(f"  Total windows: {len(dataset)}")
        print(f"  Vocabulary size: {len(tokenizer)}\n")

        for doc_idx, text in enumerate(texts):
            print(f"Document {doc_idx}:")
            print(f"  Original text: '{text}'")
            print(f"  Original length: {len(text.split())}")

            window_indices = dataset.get_document_windows(doc_idx)
            print(f"  Number of windows: {len(window_indices)}")

            for i, window_idx in enumerate(window_indices):
                tokens, attention_mask, _ = dataset[window_idx]
                # Remove the batch dimension for decoding
                decoded = tokenizer.decode(tokens.squeeze(0).tolist())
                print(f"\n    Window {i}:")
                print(f"    Tokens shape: {tokens.shape}")
                print(f"    Tokens: {tokens.squeeze(0).tolist()}")
                print(f"    Attention mask shape: {attention_mask.shape}")
                print(f"    Attention mask: {attention_mask.squeeze(0).tolist()}")
                print(f"    Decoded: '{decoded}'")
                print(f"    Window length: {tokens.size(1)}")  # Use size(1) for sequence length

            print("\n" + "-"*50)

        tokenizer.show_vocab()

    except Exception as e:
        print(f"An error occurred: {str(e)}")

    print("\n--- End of Test ---")

# Run the test
test_sliding_window_dataset()

## 📝 Positional Encoding

Positional Encoding adds information about the position of each token in the sequence. This is necessary because the self-attention mechanism in Transformers doesn't inherently have a notion of token order.


\begin{equation}
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{model}}}}\right)
\end{equation}

\begin{equation}
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{model}}}}\right)
\end{equation}



**✨ Additional Resources:**

*   Transformer Architecture: The Positional Encoding [Link-kazemnejad](https://kazemnejad.com/blog/transformer_architecture_positional_encoding/)

*   Positional Encoding in Transformers [Link-geeksforgeeks](https://www.geeksforgeeks.org/positional-encoding-in-transformers/)




In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        """
        Inputs
            d_model - Hidden dimensionality of the input.
            max_len - Maximum length of a sequence to expect.
        """
        super().__init__()

        #######################Code Here###############################

        # Create matrix of [SeqLen, HiddenDim] representing the positional encoding for max_len inputs
        ## pe = ...

        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Compute the `div_term` for the frequency component using the formula involving d_model
        ## div_term =


        # Populate the even indices (0, 2, 4, ...) of `pe` with the sine of the position multiplied by the `div_term`
        ## pe[:, 0::2] =


        # Populate the odd indices (1, 3, 5, ...) of `pe` with the cosine of the position multiplied by the `div_term`


        # Add an *extra dimension* to `pe` to fit the batch dimension requirements


        # Register `pe` as a buffer to ensure it is not considered a model parameter but is still moved to the appropriate device when the model is moved


        ###############################################################

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

### ✍ Testing your Positional Encoding Function

In [ ]:
def test_positional_encoding_consistency():
    # Test parameters
    d_model = 4
    max_len = 10
    batch_size = 2
    seq_len = 3

    # Initialize a new PositionalEncoding module each time
    pe = PositionalEncoding(d_model, max_len)

    # Create a small input tensor
    x1 = torch.zeros(batch_size, seq_len, d_model)

    # Run the positional encoding
    output = pe(x1)

    print("Input:")
    print(x1)
    print("\nOutput:")
    print(output)

    print("""\nNote: If your
    Output:
        tensor([[[ 0.0000,  1.0000,  0.0000,  1.0000],
                [ 0.8415,  0.5403,  0.0100,  0.9999],
                [ 0.9093, -0.4161,  0.0200,  0.9998]],

                [[ 0.0000,  1.0000,  0.0000,  1.0000],
                [ 0.8415,  0.5403,  0.0100,  0.9999],
                [ 0.9093, -0.4161,  0.0200,  0.9998]]])
             , the PositionalEncoding is consistent.""")

test_positional_encoding_consistency()

### ✍ Testing the Positional Encoding for Text

In [ ]:
def test_positional_encoding_with_dataset():
    print("\n--- Testing Positional Encoding with Dataset ---\n")

    # Set a fixed seed for reproducibility
    torch.manual_seed(42)

    # Sample texts
    texts = [
        "This is a short sentence.",
        "This is a much longer sentence that will be split into multiple windows. Observe the term overlapping?",
        "Another sentence of medium length."
    ]

    # Initialize tokenizer and fit it to the texts
    tokenizer = SimpleTokenizer()
    tokenizer.fit(texts)

    # Create dataset
    max_length = 10
    overlap = 2
    dataset = TextDataset(texts, tokenizer, max_length, overlap)

    # Initialize positional encoding
    d_model = 16  # Small dimension for demonstration
    pos_encoder = PositionalEncoding(d_model, max_length)

    print(f"Dataset configuration:")
    print(f"  Max length: {max_length}")
    print(f"  Overlap: {overlap}")
    print(f"  Total windows: {len(dataset)}")
    print(f"  Vocabulary size: {len(tokenizer)}")
    print(f"  Embedding dimension: {d_model}\n")

    # Process each window through the positional encoding
    for i in range(len(dataset)):
        tokens, attention_mask, doc_idx = dataset[i]

        # Convert tokens to "embeddings" (just for demonstration)
        pseudo_embeddings = torch.rand(1, tokens.size(1), d_model)  # (batch_size, seq_len, d_model)

        # Apply positional encoding
        encoded = pos_encoder(pseudo_embeddings)

        print(f"Window {i} (from document {doc_idx}):")
        print(f"  Original tokens: {tokens.squeeze(0).tolist()}")
        print(f"  Attention mask: {attention_mask.squeeze(0).tolist()}")
        print(f"  Decoded: '{tokenizer.decode(tokens.squeeze(0).tolist())}'")
        print(f"  Shape after positional encoding: {encoded.shape}")

        # Display the positional encoding effect for all tokens
        print(f"  Positional encoding effect:")
        for j in range(tokens.size(1)):
            if attention_mask[0, j] == 1:  # Only show for non-padding tokens
                print(f"    Token {j}:")
                print(f"      Before: {pseudo_embeddings[0, j, :].tolist()}")
                print(f"      After:  {encoded[0, j, :].tolist()}")

        print()

    print("--- End of Test ---")

# Run the test
test_positional_encoding_with_dataset()

## 📝 Masked Multihead Attention Mechanism

As you have implemented the Attention Mechanism in Part I, here you will have to use its ready implementation from Pytorch.

Masked Attention mechanism allows the transformer model to focus on relevant parts of the input sequence while preventing information leakage from future tokens during sequential processing (i.e. we use the term Masked).

Please, visit [Link-pytorch](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) for a detailed description of the MultiheadAttention function in PyTorch.

**✨ Additional Resources:**

*   Multi-head Attention, deep dive [Link-towardsdatascience](https://towardsdatascience.com/transformers-explained-visually-part-3-multi-head-attention-deep-dive-1c1ff1024853)
* Attention Is All You Need (original Transformer paper) [Link-ArXiv](https://arxiv.org/abs/1706.03762)

* A visual explanation of the attention mechanism [Link-youtube](https://www.youtube.com/watch?v=bCz4OMemCcA&t=1208s&ab_channel=UmarJamil)

In [ ]:
class MaskedAttention(nn.Module):
    def __init__(self, d_model: int, nhead: int, dropout: float = 0.1):
        super().__init__()

        #######################Code Here###############################

        # Initialize the MultiheadAttention layer using its PyTorch implementation

        # Initialize the parameters

        ###############################################################


    def generate_square_subsequent_mask(self, sz: int) -> torch.Tensor:

        #######################Code Here###############################
        # Create a triangular mask tensor in PyTorch where the lower triangle (including the diagonal) is
        # filled with ones and the upper triangle is filled with zeros. The tensor should be of size (sz, sz),
        # where sz is a given integer.
        ##############################################################

        # The masked positions are filled with float('-inf'). Unmasked positions are filled with float(0.0). See: https://pytorch.org/docs/stable/_modules/torch/nn/modules/transformer.html#Transformer
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        seq_len = x.size(1)
        attn_mask = self.generate_square_subsequent_mask(seq_len).to(x.device)
        output, _ = self.multihead_attn(x, x, x, attn_mask=attn_mask)
        return output, attn_mask

### ✍ Testing the Multihead Attention

In [ ]:
def test_expanded_transformer_components():
    print("\n--- Testing Expanded Transformer Components ---\n")

    # Set a fixed seed for reproducibility
    torch.manual_seed(42)

    # Sample texts
    texts = [
        "This is a short sentence.",
        "This is a much longer sentence that will be split into multiple windows. Observe the term overlapping?",
        "Another sentence of medium length."
    ]

    # Initialize tokenizer and fit it to the texts
    tokenizer = SimpleTokenizer()
    tokenizer.fit(texts)

    # Create dataset
    max_length = 10
    overlap = 2
    dataset = TextDataset(texts, tokenizer, max_length, overlap)

    # Hyperparameters
    d_model = 16  # Small dimension for demonstration
    nhead = 2

    # Initialize components
    pos_encoder = PositionalEncoding(d_model, max_length)
    masked_self_attn = MaskedAttention(d_model, nhead)

    print(f"Dataset configuration:")
    print(f"  Max length: {max_length}")
    print(f"  Overlap: {overlap}")
    print(f"  Total windows: {len(dataset)}")
    print(f"  Vocabulary size: {len(tokenizer)}")
    print(f"  Embedding dimension: {d_model}")
    print(f"  Number of attention heads: {nhead}\n")

    # Process each window
    for i in range(len(dataset)):
        tokens, attention_mask, doc_idx = dataset[i]

        print(f"Window {i} (from document {doc_idx}):")
        print(f"  Original tokens: {tokens.squeeze(0).tolist()}")
        print(f"  Attention mask: {attention_mask.squeeze(0).tolist()}")
        print(f"  Decoded: '{tokenizer.decode(tokens.squeeze(0).tolist())}'")

        # Convert tokens to "embeddings" (just for demonstration)
        pseudo_embeddings = torch.rand(1, tokens.size(1), d_model)  # (batch_size, seq_len, d_model)
        print(f"  Shape of pseudo embeddings: {pseudo_embeddings.shape}")

        # Apply positional encoding
        pos_encoded = pos_encoder(pseudo_embeddings)
        print(f"  Shape after positional encoding: {pos_encoded.shape}")

        # Apply masked self-attention
        attn_output, attn_mask = masked_self_attn(pos_encoded)
        print(f"  Shape after masked self-attention: {attn_output.shape}")

        # Display the effect of positional encoding and attention for all tokens
        print(f"  Transformer effect on tokens:")
        for j in range(tokens.size(1)):
            if attention_mask[0, j] == 1:  # Only show for non-padding tokens
                print(f"    Token {j}:")
                print(f"      Initial:   {pseudo_embeddings[0, j, :5].tolist()}")
                print(f"      Positional:{pos_encoded[0, j, :5].tolist()}")
                print(f"      Attention Mask: {attn_mask[j, :5].tolist()}")  # Show first 5 values of attention mask
                print(f"      Attention: {attn_output[0, j, :5].tolist()}")
        print()

    print("--- End of Expanded Test ---")

# Run the expanded test
test_expanded_transformer_components()

## 📝 Feed Forward Netwrok

A feed-forward network is a multi-layered structure in which information moves in a single direction, from the input layer to the output layer.


**✨ Additional Resources:**

*   Transformer Feed-Forward Layers Are Key-Value Memories
 [Link-ArXiv](https://arxiv.org/abs/2012.14913)







In [ ]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super().__init__()

        #######################Code Here###############################
        # Define feed-forward network using nn.Sequential
        self.net = nn.Sequential(
          # First linear layer
          # ReLU activation
          # Dropout for regularization
          # Second linear layer
        )
        ###############################################################


    def forward(self, x):
        return self.net(x)  # Apply the feed-forward network

### ✍ Testing the Feed Forward Layer

In [ ]:
def test_all_transformer_components():
    print("\n--- Testing Transformer Components ---\n")

    # Set a fixed seed for reproducibility
    torch.manual_seed(42)

    # Sample texts
    texts = [
        "This is a short sentence.",
        "This is a much longer sentence that will be split into multiple windows. Observe the term overlapping?",
        "Another sentence of medium length."
    ]

    # Initialize tokenizer and fit it to the texts
    tokenizer = SimpleTokenizer()
    tokenizer.fit(texts)

    # Create dataset
    max_length = 10
    overlap = 2
    dataset = TextDataset(texts, tokenizer, max_length, overlap)

    # Hyperparameters
    d_model = 16  # Small dimension for demonstration
    nhead = 2
    d_ff = d_model  # Feed-forward dimension
    dropout = 0.1

    # Initialize components
    pos_encoder = PositionalEncoding(d_model, max_length)
    masked_self_attn = MaskedAttention(d_model, nhead)
    feed_forward = FeedForward(d_model, d_ff, dropout)

    print(f"Dataset configuration:")
    print(f"  Max length: {max_length}")
    print(f"  Overlap: {overlap}")
    print(f"  Total windows: {len(dataset)}")
    print(f"  Vocabulary size: {len(tokenizer)}")
    print(f"  Embedding dimension: {d_model}")
    print(f"  Number of attention heads: {nhead}")
    print(f"  Feed-forward dimension: {d_ff}\n")

    # Process each window
    for i in range(len(dataset)):
        tokens, attention_mask, doc_idx = dataset[i]

        print(f"Window {i} (from document {doc_idx}):")
        print(f"  Original tokens: {tokens.squeeze(0).tolist()}")
        print(f"  Attention mask: {attention_mask.squeeze(0).tolist()}")
        print(f"  Decoded: '{tokenizer.decode(tokens.squeeze(0).tolist())}'")

        # Convert tokens to "embeddings" (just for demonstration)
        pseudo_embeddings = torch.rand(1, tokens.size(1), d_model)  # (batch_size, seq_len, d_model)
        print(f"  Shape of pseudo embeddings: {pseudo_embeddings.shape}")

        # Apply positional encoding
        pos_encoded = pos_encoder(pseudo_embeddings)
        print(f"  Shape after positional encoding: {pos_encoded.shape}")

        # Apply masked self-attention
        attn_output, attn_mask = masked_self_attn(pos_encoded)
        print(f"  Shape after masked self-attention: {attn_output.shape}")

        # Apply feed-forward network
        ff_output = feed_forward(attn_output)
        print(f"  Shape after feed-forward: {ff_output.shape}")

        # Display the effect of positional encoding, attention, and feed-forward for all tokens
        print(f"  Transformer effect on tokens:")
        for j in range(tokens.size(1)):
            if attention_mask[0, j] == 1:  # Only show for non-padding tokens
                print(f"    Token {j}:")
                print(f"      Initial:   {pseudo_embeddings[0, j, :5].tolist()}")
                print(f"      Positional:{pos_encoded[0, j, :5].tolist()}")
                print(f"      Attention Mask: {attn_mask[j, :5].tolist()}")  # Show first 5 values
                print(f"      Attention: {attn_output[0, j, :5].tolist()}")
                print(f"      Feed-Forward: {ff_output[0, j, :5].tolist()}")
        print()

    print("--- End of Expanded Test ---")

# Run the expanded test
test_all_transformer_components()

## 📝 Decoder Layer

Implementation of a Transformer Decoder Layer with Masked Multi-Head Attention and Feed Forward Network








![Decoder Only Architecture](https://drive.google.com/uc?id=1ksROxQxf3b7dlBUoIQggzyLeBaPO-AQn)




In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model: int, nhead: int, d_ff: int, dropout: float = 0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(d_model)

        #######################Code Here###############################

        # Define the decoder - Layer Normalization, Attention, Feed_forward Dropouts

        ###############################################################

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Masked Multi-Head Attention
        normed_x = self.norm1(x)
        attn_output, _ = self.masked_attention(normed_x) # _ because we returned also the mask in the previous demonstration

        #######################Code Here###############################.

        # Create the remaining connection and residual connections - What is a residual connection? (https://towardsdatascience.com/what-is-residual-connection-efb07cab0d55)

        ###############################################################

        return x

## 📚 Decoder-only Transformer

The components of a Decoder-Only Transformer include an embedding layer for token representation, positional encoding for sequential information, stacked decoder layers for hierarchical processing, layer normalization for stability, and an output projection layer for generating tokens.

In [ ]:
class DecoderOnlyTransformer(nn.Module):
    def __init__(self, vocab_size: int, d_model: int, nhead: int, num_layers: int,
                 d_ff: int, max_seq_length: int, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.max_seq_length = max_seq_length

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, d_model)

        # Positional encoding
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        # Decoder layers
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, nhead, d_ff, dropout)
            for _ in range(num_layers)
        ])

        # Final layer norm
        self.final_norm = nn.LayerNorm(d_model)

        # Output projection
        self.output_projection = nn.Linear(d_model, vocab_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: (batch_size, seq_len)

        # Embed the input
        x = self.embedding(x) * math.sqrt(self.d_model)

        # Add positional encoding
        x = self.positional_encoding(x)

        # Apply decoder layers
        for layer in self.layers:
            x = layer(x)

        # Apply final layer norm
        x = self.final_norm(x)

        # Project to vocabulary size
        output = self.output_projection(x)

        return output

    def generate(self, start_tokens: torch.Tensor, max_length: int,
                 temperature: float = 1.0) -> torch.Tensor:
        self.eval()
        current_seq = start_tokens

        with torch.no_grad():
            for _ in range(max_length - start_tokens.size(1)):
                # Ensure we're not exceeding the maximum sequence length
                if current_seq.size(1) > self.max_seq_length:
                    current_seq = current_seq[:, -self.max_seq_length:]

                # Get model predictions
                logits = self(current_seq)
                next_token_logits = logits[:, -1, :] / temperature

                # Sample next token
                probs = F.softmax(next_token_logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)

                # Append next token to sequence
                current_seq = torch.cat([current_seq, next_token], dim=1)

                # Check if we've generated an EOS token
                if next_token.item() == self.vocab_size - 1:  # Assuming EOS is the last token
                    break

        return current_seq

### ✍ Displaying the Decoder-only Transformer Architecture

In [ ]:
!pip install torchinfo
from torchinfo import summary

# Initialize the model with some example parameters
vocab_size = 10000
d_model = 512
nhead = 2
num_layers = 1
d_ff = 2048
max_seq_length = 1024
dropout = 0.1

# Define your model
model = DecoderOnlyTransformer(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout
)

# Print the model summary
summary(model, input_size=(1, max_seq_length), dtypes=[torch.int64])

## 📚 Training the Decoder-only Transformer

In [ ]:
# Load the tiny_shakespeare dataset
dataset = load_dataset("tiny_shakespeare", split="train")
# Load the tiny_shakespeare dataset
# dataset = load_dataset("lyimo/shakespear", split="train")

# Extract the text from the dataset
texts = dataset["text"]

# Hyperparameters
d_model = 256
nhead = 2
num_layers = 2
d_ff = 256
max_seq_length = 128
batch_size = 64
num_epochs = 10
learning_rate = 0.0001
dropout = 0.2

# Tokenize and prepare data
tokenizer = SimpleTokenizer()
tokenizer.fit(texts)
vocab_size = len(tokenizer.word_to_idx)

dataset = TextDataset(texts, tokenizer, max_seq_length)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Vocabulary size: {vocab_size}")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model and move to device
model = DecoderOnlyTransformer(vocab_size, d_model, nhead, num_layers, d_ff, max_seq_length, dropout).to(device)

# Create optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.word_to_idx["<PAD>"])

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_seq, _, _ = batch  # Unpack batch
        input_seq = input_seq.squeeze(1).to(device)  # Move input to device and remove extra dimension

        # Forward pass
        output = model(input_seq)

        # Reshape output tensor
        output = output[:, :-1, :].contiguous().view(-1, output.size(-1))  # Shift predictions to the left

        # Shift targets to the right (original targets)
        target_seq = input_seq[:, 1:].contiguous().view(-1)

        # Compute loss
        loss = criterion(output, target_seq)

        # Debugging prints
        print(f"Loss: {loss.item()}")

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx == 0:
          # Debugging prints
          print(f"Epoch: {epoch+1}, Batch: {batch_idx+1}")
          print(f"Input sequence shape: {input_seq.shape}")
          print(f"Input sequence: {input_seq.unsqueeze(1)}")
          print(f"Output shape before reshape: {output.shape}")
          print(f"Output shape after reshape: {output.shape}")
          print(f"Target sequence shape: {target_seq.shape}")

    # Print epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")

### ✍ Testing the Decoder-only Transformer

In [ ]:
texts = ["Better three hours too soon than", " I believe I can ", "My words fly up, my", "Brevity is ", "Love looks not with the eyes, but", "To be or "]

for quote in texts:
  start_tokens = torch.tensor(tokenizer.encode(quote)).unsqueeze(0).to(device)  # Add batch dimension and move to device

  generated_tokens = model.generate(start_tokens, max_length=20, temperature=.9)
  generated_text = tokenizer.decode(generated_tokens.squeeze().tolist())

  print(generated_text)

_____________________________________________

# Decoder-only with MoE instead of FFN

In the Sparse Mixture of Experts (MoE) architecture, the self-attention mechanism within each transformer block stays the same.

However, a key modification is made to the structure **of each block**: the standard **feed-forward neural network** is replaced with **multiple sparsely activated feed-forward networks, known as experts.**

"Sparse activation" means that each token in the sequence is routed to only a small number of these experts—usually one or two—out of the entire pool.



**✨ Additional Resources:**

*   makeMoE: Implement a Sparse Mixture of Experts Language Model from Scratch [Link-huggingface](https://huggingface.co/blog/AviSoori1x/makemoe-from-scratch)




_____________________________________________

## 📝 Expert Layer

In [ ]:
class Expert(nn.Module):
    """ An MLP with a single hidden layer and ReLU activation, serving as an expert in a Mixture of Experts. """
    def __init__(self, n_embd: int, dropout: float = 0.1):
        super().__init__()
        self.net = nn.Sequential(

        #######################Code Here###############################

        # Define a Linear, ReLU, Linaer and Dropout, Linear should be (n_embd, 4 * n_embd)

        ###############################################################

      )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)

## 📚 Gating in MoE Architectures

Types of gating in Mixture of Experts (MoE) systems include Top-k gating, Noisy Top-k gating (as implemented here), and other variants like Hierarchical gating or Soft gating.

Gating is essential in MoE systems because it determines which experts to use for each input, allowing the model to specialize different experts for different types of inputs or tasks.

Specifically, Noisy Top-k gating adds controlled randomness to the expert selection process, which can help balance expert utilization and potentially improve model performance by introducing exploration in the routing mechanism.

In [ ]:
class NoisyTopkRouter(nn.Module):
    def __init__(self, n_embed, num_experts, top_k_moe):
        super(NoisyTopkRouter, self).__init__()
        # Store the top_k_moe parameter which specifies the number of top experts to select
        self.top_k_moe = top_k_moe
        # Linear layer to compute logits for routing
        self.topkroute_linear = nn.Linear(n_embed, num_experts)
        # Linear layer to compute noise logits for added noise
        self.noise_linear = nn.Linear(n_embed, num_experts)

    def forward(self, mh_output):
        # Compute the logits for routing to experts
        logits = self.topkroute_linear(mh_output)
        # Compute the noise logits
        noise_logits = self.noise_linear(mh_output)
        # Generate noise with standard deviation determined by softplus of noise logits
        noise = torch.randn_like(logits) * F.softplus(noise_logits)
        # Add noise to the original logits to get noisy logits
        noisy_logits = logits + noise
        # Select the top k logits and their indices from the noisy logits
        top_k_moe_logits, indices = noisy_logits.topk(self.top_k_moe, dim=-1)
        # Create a tensor full of -inf values
        zeros = torch.full_like(noisy_logits, float('-inf'))
        # Scatter the top k logits into the zeros tensor to create a sparse logits tensor
        sparse_logits = zeros.scatter(-1, indices, top_k_moe_logits)
        # Apply softmax to the sparse logits to get the final router output
        router_output = F.softmax(sparse_logits, dim=-1)
        # Return the router output and the indices of the selected experts
        return router_output, indices

## 📚 Sparse MoE Layer

In [ ]:
class SparseMoE(nn.Module):
    def __init__(self, n_embed, num_experts, top_k_moe):
        super(SparseMoE, self).__init__()
        # Initialize the NoisyTopkRouter to determine which experts to activate
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k_moe)
        # Create a list of expert networks, each being a feed-forward network
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        # Store the number of top experts to activate
        self.top_k_moe = top_k_moe

    def forward(self, x):
        # Get the gating output and indices from the router
        gating_output, indices = self.router(x)
        # Initialize the final output tensor with zeros, having the same shape as x
        final_output = torch.zeros_like(x)
        # Flatten the input tensor to simplify processing
        flat_x = x.view(-1, x.size(-1))
        # Flatten the gating output tensor to align with the flattened input
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        # Iterate over each expert
        for i, expert in enumerate(self.experts):
            # Create a mask to identify where the current expert is used
            expert_mask = (indices == i).any(dim=-1)
            # Flatten the expert mask to match the flattened input
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():  # Check if there are any positions using the current expert
                # Extract the inputs for the current expert based on the mask
                expert_input = flat_x[flat_mask]
                # Get the output from the current expert
                expert_output = expert(expert_input)
                # Get the gating scores for the current expert
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                # Compute the weighted output based on the gating scores
                weighted_output = expert_output * gating_scores
                # Add the weighted output to the final output tensor
                final_output[expert_mask] += weighted_output.view_as(final_output[expert_mask])

        # Return the final output tensor which combines the results from all activated experts
        return final_output


## 📝 Decoder with Sparse MoE

![Decoder Only Architecture](https://drive.google.com/uc?id=1ksROxQxf3b7dlBUoIQggzyLeBaPO-AQn)



In [ ]:
class DecoderLayerMoE(nn.Module):
    def __init__(self, d_model, nhead, d_ff, num_experts, top_k_moe, dropout=0.1):
        super().__init__()
        #######################Code Here###############################

        # Create the Decoder architecture as before, but now add the MoE block instead of the FFN


    def forward(self, x):

        return x

        ###############################################################

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

### ✍ Showcase how Sparse MoE handles its inputs

-  Experiment: Change the number of experts in the SparseMoE_example model.
  -  Observation: Observe how increasing or decreasing the number of experts affects the routing, gating outputs, and final output.

- Experiment: Adjust the top_k_moe parameter to select more or fewer top experts.
  - Observation: See how the number of experts activated for each token changes and how it impacts the final output.



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SparseMoE_example(nn.Module):
    def __init__(self, n_embed, num_experts, top_k_moe):
        super(SparseMoE_example, self).__init__()
        self.router = NoisyTopkRouter(n_embed, num_experts, top_k_moe)
        self.experts = nn.ModuleList([Expert(n_embed) for _ in range(num_experts)])
        self.top_k_moe = top_k_moe

    def forward(self, x):
        gating_output, indices = self.router(x)
        final_output = torch.zeros_like(x)
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        for i, expert in enumerate(self.experts):
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                expert_output = expert(expert_input)
                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                weighted_output = expert_output * gating_scores
                final_output[expert_mask] += weighted_output.view_as(final_output[expert_mask])

        return final_output

    def forward_debug_example(self, x):
        # Forward pass with debug prints
        gating_output, indices = self.router(x)
        print("Gating Output Shape:", gating_output.shape)
        print("Gating Output:", gating_output)
        print("Expert Indices Shape:", indices.shape)
        print("Expert Indices:", indices)

        print("Input Shape:", x.shape)
        print("Input:", x)

        final_output = torch.zeros_like(x)
        flat_x = x.view(-1, x.size(-1))
        flat_gating_output = gating_output.view(-1, gating_output.size(-1))

        for i, expert in enumerate(self.experts):
            print("\n" + "-"*50)
            expert_mask = (indices == i).any(dim=-1)
            flat_mask = expert_mask.view(-1)

            if flat_mask.any():
                expert_input = flat_x[flat_mask]
                print(f"Expert {i} Input Shape:", expert_input.shape)
                print(f"Expert {i} Input:", expert_input)

                expert_output = expert(expert_input)
                print(f"Expert {i} Output Shape:", expert_output.shape)
                print(f"Expert {i} Output:", expert_output)

                gating_scores = flat_gating_output[flat_mask, i].unsqueeze(1)
                print(f"Gating Scores for Expert {i}:")
                print(gating_scores.squeeze())

                print(f"Weighted Output for Expert {i}:")
                weighted_output = expert_output * gating_scores
                print(weighted_output)

                final_output[expert_mask] += weighted_output.view_as(final_output[expert_mask])
                print(f"Expert {i} final_output Shape:", final_output[expert_mask].shape)
                print(f"Expert {i} final_output:", final_output[expert_mask])

        print("Final MoE Output Shape:", final_output.shape)
        print("Final MoE Output:", final_output)
        print("-"*50)
        return final_output


# Example usage and debugging prints
def test_sparse_moe():
    # Parameters
    batch_size = 2   #
    seq_length = 1   # number of tokens, if 1 then it is easier to see which experts are activated and how each embedding is calculated
    n_embed = 5
    num_experts = 6  # Increased number of experts
    top_k_moe = 2   # If you modify, more or less experts will be activated for each input token

    # Random input tensor (simulating token embeddings)
    random_input = torch.randn(batch_size, seq_length, n_embed)

    # Initialize SparseMoE
    sparse_moe = SparseMoE_example(n_embed, num_experts, top_k_moe)

    # Forward pass with debugging example
    final_output = sparse_moe.forward_debug_example(random_input)

    print("\nRandom Input Tensor:")
    print(random_input)
    print("\nFinal Output Tensor (after MoE processing):")
    print(final_output)

# Run the test function
test_sparse_moe()

## 📚 Decoder-only Transformer with MoE

In [ ]:
class DecoderOnlyTransformerMoE(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, d_ff, max_seq_length, dropout, num_experts, top_k_moe):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_seq_length)
        self.layers = nn.ModuleList([DecoderLayerMoE(d_model, nhead, d_ff, num_experts, top_k_moe, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        self.output = nn.Linear(d_model, vocab_size)
        self.max_seq_length = max_seq_length
        self.num_experts = num_experts
        self.top_k_moe = top_k_moe
        self.vocab_size = vocab_size

    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)
        return self.output(x)

    def generate(self, start_tokens: torch.Tensor, max_length: int, temperature: float = 1.0) -> torch.Tensor:
        self.eval()
        current_seq = start_tokens

         with torch.no_grad():  # Disable gradient computation
            # Generate tokens until max_length is reached or end token is generated
            for _ in range(max_length - start_tokens.size(1)):
                # Ensure the sequence length does not exceed max_seq_length
                if current_seq.size(1) > self.max_seq_length:
                    current_seq = current_seq[:, -self.max_seq_length:]

                # Get logits from the model
                logits = self(current_seq)

                # Extract logits for the next token and scale by temperature
                next_token_logits = logits[:, -1, :] / temperature

                # Compute probabilities using softmax
                probs = F.softmax(next_token_logits, dim=-1)

                # Sample the next token from the probability distribution
                next_token = torch.multinomial(probs, num_samples=1)

                # Append the next token to the current sequence
                current_seq = torch.cat([current_seq, next_token], dim=1)

                # Stop if the end token is generated (vocab_size - 1 assumed to be the end token)
                if next_token.item() == self.vocab_size - 1:
                    break

        # Return the generated sequence
        return current_seq

### ✍ Displaying the Decoder-only Transformer with MoE Architecture

In [ ]:
# Initialize the model with some example parameters
vocab_size = 10000
d_model = 512
nhead = 2
num_layers = 1
d_ff = 2048
max_seq_length = 1024
dropout = 0.1
num_experts = 4
top_k_moe = 2

# Define your model
model = DecoderOnlyTransformer(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout,
)

# Define your model
model_moe = DecoderOnlyTransformerMoE(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout,
    num_experts=num_experts,
    top_k_moe=top_k_moe
)

# Print the model summary
print(50*"-")
print(summary(model, input_size=(1, max_seq_length), dtypes=[torch.int64]))
print(50*"-")
print(summary(model_moe, input_size=(1, max_seq_length), dtypes=[torch.int64]))

In [ ]:
def print_model_summary(model, input_size):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    dummy_input = torch.zeros(input_size, dtype=torch.int64).to(device)

    def register_hook(module):
        def hook(module, input, output):
            class_name = module.__class__.__name__
            module_idx = len(summary)
            m_key = f"{module_idx:03d} {class_name}"
            summary[m_key] = {}
            summary[m_key]["input_shape"] = list(input[0].size())
            if isinstance(output, torch.Tensor):
                summary[m_key]["output_shape"] = list(output.size())
            elif isinstance(output, (tuple, list)) and len(output) > 0 and isinstance(output[0], torch.Tensor):
                summary[m_key]["output_shape"] = [list(out.size()) for out in output]
            else:
                summary[m_key]["output_shape"] = "multiple outputs"
            params = sum(p.numel() for p in module.parameters())
            summary[m_key]["num_params"] = params

        if not isinstance(module, nn.Sequential) and not isinstance(module, nn.ModuleList):
            hooks.append(module.register_forward_hook(hook))

    summary = {}
    hooks = []
    model.apply(register_hook)
    model(dummy_input)
    for h in hooks:
        h.remove()

    print("----------------------------------------------------------------")
    print("{:>20}  {:>25} {:>15}".format("Layer (type)", "Input Shape", "Param #"))
    print("================================================================")
    total_params = 0
    total_output = 0
    for layer in summary:
        line_new = "{:>20}  {:>25} {:>15}".format(
            layer,
            str(summary[layer]["input_shape"]),
            "{0:,}".format(summary[layer]["num_params"]),
        )
        total_params += summary[layer]["num_params"]
        if isinstance(summary[layer]["output_shape"], list) and all(isinstance(i, int) for i in summary[layer]["output_shape"]):
            total_output += np.prod(summary[layer]["output_shape"])
        print(line_new)
    print("================================================================")
    print(f"Total params: {total_params:,}")
    print("----------------------------------------------------------------")

vocab_size = 10000
d_model = 512
nhead = 8
num_layers = 1
d_ff = 2048
max_seq_length = 1024
dropout = 0.1
num_experts = 2
top_k_moe = 1

# Ensure you have the correct definitions for these classes
# from your_model_definitions import DecoderOnlyTransformer, DecoderOnlyTransformerMoE

model = DecoderOnlyTransformer(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout,
)

model_moe = DecoderOnlyTransformerMoE(
    vocab_size=vocab_size,
    d_model=d_model,
    nhead=nhead,
    num_layers=num_layers,
    d_ff=d_ff,
    max_seq_length=max_seq_length,
    dropout=dropout,
    num_experts=num_experts,
    top_k_moe=top_k_moe
)

print("Summary for DecoderOnlyTransformer")
print_model_summary(model, (1, max_seq_length))

print("\nSummary for DecoderOnlyTransformerMoE")
print_model_summary(model_moe, (1, max_seq_length))


## 📚 Training the Decoder-only Transformer with MoE

In [ ]:
# Load the tiny_shakespeare dataset
dataset = load_dataset("tiny_shakespeare", split="train")

# Extract the text from the dataset
texts = dataset["text"]

# Hyperparameters
d_model = 128
nhead = 2
num_layers = 2
d_ff = 256
max_seq_length = 64
batch_size = 32
num_epochs = 1
learning_rate = 0.0001
dropout = 0.2
num_experts=4
top_k_moe=2

# Tokenize and prepare data
tokenizer = SimpleTokenizer()
tokenizer.fit(texts)
vocab_size = len(tokenizer.word_to_idx)

dataset = TextDataset(texts, tokenizer, max_seq_length)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

print(f"Vocabulary size: {vocab_size}")

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model and move to device
model_moe = DecoderOnlyTransformerMoE(vocab_size, d_model, nhead, num_layers, d_ff, max_seq_length, dropout, num_experts, top_k_moe).to(device)

# Create optimizer and loss function
optimizer = torch.optim.AdamW(model_moe.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.word_to_idx["<PAD>"])

# Training loop
for epoch in range(num_epochs):
    model_moe.train()
    total_loss = 0
    for batch_idx, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_seq, _, _ = batch  # Unpack batch
        input_seq = input_seq.squeeze(1).to(device)  # Move input to device and remove extra dimension

        # Forward pass
        output = model_moe(input_seq)


        # Reshape output tensor
        output = output[:, :-1, :].contiguous().view(-1, output.size(-1))  # Shift predictions to the left

        # Shift targets to the right (original targets)
        target_seq = input_seq[:, 1:].contiguous().view(-1)


        # Compute loss
        loss = criterion(output, target_seq)

        # Debugging prints
        print(f"Loss: {loss.item()}")

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_idx == 0:
          # Debugging prints
          print(f"Epoch: {epoch+1}, Batch: {batch_idx+1}")
          print(f"Input sequence shape: {input_seq.shape}")
          print(f"Input sequence: {input_seq.unsqueeze(1)}")
          print(f"Output shape before reshape: {output.shape}")
          print(f"Output shape after reshape: {output.shape}")
          print(f"Target sequence shape: {target_seq.shape}")

    # Print epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.4f}")


### ✍ Testing the Decoder-only Transformer with MoE

In [ ]:
texts = ["Better three hours too soon than", " I believe I can ", "My words fly up, my", "Brevity is ", "Love looks not with the eyes, but", "To be or "]

for quote in texts:
  start_tokens = torch.tensor(tokenizer.encode(quote)).unsqueeze(0).to(device)  # Add batch dimension and move to device

  generated_tokens = model_moe.generate(start_tokens, max_length=20, temperature=.9)
  generated_text = tokenizer.decode(generated_tokens.squeeze().tolist())

  print(generated_text)